# Clasificación de municipios (Similares)

Un municipio puede ser “similar” a otro según varias dimensiones, como puede ser su Índice global (aunque esta es demasiado general). En este caso se emplean los vectores de categorías (salud, educación, bienestar, economía…). Municipios similares serán los que tengan perfiles parecidos en estas dimensiones.

Los motivos son los siguientes:

- Están normalizadas (0–1).

- Resumen información real del municipio.

- Son robustas a ruido.

- Son interpretables (cuando veas un cluster sabrás por qué están juntos).

Así que el clustering se hará sobre estas 6 columnas.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

In [2]:
# Cargar el conjunto de datos
df = pd.read_csv("../website_data/df_indices_categorias.csv")

feature_cols = ["sanidad", "educacion", "transporte_conectividad", "economia_empleo", "bienestar", "ocio_cultura", 'servicios']
X = df[feature_cols].copy()
municipios = df["municipio"]

print(X.head())

    sanidad  educacion  transporte_conectividad  economia_empleo  bienestar  \
0  0.000000   0.979201                 0.724337         0.858728   0.341189   
1  0.700159   0.556944                 0.000000         0.828036   0.508173   
2  0.150871   0.718761                 0.365968         0.530654   0.146520   
3  0.511035   0.628643                 0.147945         0.552960   0.823978   
4  0.449325   0.614816                 0.090893         0.402615   0.832984   

   ocio_cultura  servicios  
0      0.125203   0.000000  
1      0.703002   0.816960  
2      0.142819   0.177304  
3      0.659231   0.742179  
4      0.728355   0.682687  


## PCA

- No dependes de clusters.
- Cada municipio tendrá su propio conjunto único de similares,

In [3]:
# PCA (2 O 3 componentes funcionanmuy bien)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Matriz de distancias
dist_matrix = cosine_distances(X_pca)

# Función para obtener los municipios más similares
k = 10  # Número de municipios similares a encontrar

similares = {}

for i, muni in enumerate(municipios):
    # ordenar por distancia (de menor a mayor)
    idx = dist_matrix[i].argsort()[1:k+1]
    similares[muni] = municipios.iloc[idx].tolist()

# Convertimos a tabla
df_similares = (
    pd.DataFrame.from_dict(similares, orient="index")
      .reset_index()
)

df_similares.columns = ["municipio", "sim1", "sim2", "sim3", "sim4", "sim5", "sim6", "sim7", "sim8", "sim9", "sim10"]

df_similares.to_csv("../website_data/municipios_similares.csv", index=False)
df_similares.head()

,municipio,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
0,La Acebeda,Puebla de la Sierra,La Hiruela,Brea de Tajo,Orusco de Tajuña,El Atazar,Cervera de Buitrago,Robledillo de la Jara,Robregordo,Gascones,Puentes Viejas
1,Ajalvir,Fuente el Saz de Jarama,Casarrubuelos,Cobeña,Chinchón,Alpedrete,Becerril de la Sierra,Torrelodones,Galapagar,Colmenarejo,Morata de Tajuña
2,Alameda del Valle,Fresno de Torote,Santa María de la Alameda,Batres,Villamanrique de Tajo,Quijorna,Berzosa del Lozoya,Garganta de los Montes,Puentes Viejas,Gascones,Robregordo
3,El Álamo,Torres de la Alameda,Aldea del Fresno,Pedrezuela,El Molar,Humanes de Madrid,Villarejo de Salvanés,Lozoyuela-Navas-Sieteiglesias,Arroyomolinos,Pelayos de la Presa,Brunete
4,Aldea del Fresno,El Álamo,Villalbilla,Valverde de Alcalá,Torres de la Alameda,Perales de Tajuña,Humanes de Madrid,Pedrezuela,Moralzarzal,Villarejo de Salvanés,El Molar


¿Por qué usar distancia coseno?

Porque es más estable cuando los datos están estandarizados.

Evita que un municipio con valores más grandes parezca “más lejano”

Captura muy bien el “perfil” de un municipio